# Rigidity Calculation Logic

This notebook details the logical steps in converting a rigidity file to the rigidity results

Firstly we import the required libraries:

In [183]:
import pandas as pd
import numpy as np

Read in the data - it will need to be converted from a `.psd` file into a `.txt` file

In [184]:
data = pd.read_csv('p1350101l.txt', skiprows=2, sep='\t')

Now we reformat the data:
* Channel A is proportional to the Torque measured by the machine
* Channel B is proportional to the degree of rotation measured by the machine

In [185]:
chanb_index = data.loc[data['Time'] == 'Channel B'].index[0]
chana_df=data.iloc[1:chanb_index]
chanb_df=data.iloc[chanb_index+3:len(data)]
all_df = chana_df.copy()
all_df.columns = ['Time', 'Channel A']
all_df['Channel B'] = chanb_df[' Value'].values
all_df = all_df[['Time', 'Channel B', 'Channel A']].reset_index(drop=True)
all_df

,Time,Channel B,Channel A
0,0,-152,199
1,31,-189,245
2,62,-227,299
3,93,-256,335
4,125,-295,338
...,...,...,...
1361,45630,-630,148
1362,45661,-629,191
1363,45692,-628,210
1364,45724,-629,201


Scalings are required to recover the Torque in mNm, and the Deviation in degrees.

Torque $t$ from observations in channel $a$:
$$t = \frac{4256.6 a - 211.64}{1000} $$

Deviation $d$ from observations in channel $b$: 
$$d = \frac{40}{1400}b - \frac{40}{14} $$

In [186]:
convert_df = all_df[['Time', 'Channel B']]
convert_df.reset_index(inplace=True, drop=True)
convert_df['Channel B'] = pd.to_numeric(convert_df['Channel B'])
convert_df['Torque (mNm)'] = all_df['Channel A'].apply(lambda x: (4256.6 * float(x) - 211.64)/1000)
convert_df['Deviation (Degrees)'] = all_df['Channel B'].apply(lambda x: float(x)*(40/1400)-(40/14))
convert_df

/var/folders/qp/723wnmwx4sbbcgxfnsdr9wyr0000gn/T/ipykernel_11665/1812090070.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  convert_df['Channel B'] = pd.to_numeric(convert_df['Channel B'])


,Time,Channel B,Torque (mNm),Deviation (Degrees)
0,0,-152,846.85176,-7.200000
1,31,-189,1042.65536,-8.257143
2,62,-227,1272.51176,-9.342857
3,93,-256,1425.74936,-10.171429
4,125,-295,1438.51916,-11.285714
...,...,...,...,...
1361,45630,-630,629.76516,-20.857143
1362,45661,-629,812.79896,-20.828571
1363,45692,-628,893.67436,-20.800000
1364,45724,-629,855.36496,-20.828571


Next, blocks of observations containing significant change to the Torque observed are identified. That is where we have sets 4 consecutive observations which pairwaise differ by $>5$ or $<-5$. We collect all these blocks together.

In [187]:
convert_df['Cond1'] = False
convert_df['Cond2'] = False

# Identify blocks which satisfy the condition specified above

for i in [x+1 for x in range(len(convert_df)-3)]:
    vals = convert_df.loc[i-1:i+2,'Channel B']
    if sum(vals.diff().dropna().values<-5) == 3:
        convert_df.at[i, 'Cond1'] = True
    if sum(vals.diff().dropna().values>5) == 3:
        convert_df.at[i, 'Cond2'] = True

# Collect these blocks together in a nested list

in_block = False
block1_dct = {}
for i in range(len(convert_df)):
    val = convert_df.at[i, 'Cond1']
    if not in_block:
        if val:
            in_block = True
            index = i
    if in_block:
        if not val:
            block1_dct[index] = i
            in_block = False

in_block = False
block2_dct = {}
for i in range(len(convert_df)):
    val = convert_df.at[i, 'Cond2']
    if not in_block:
        if val:
            in_block = True
            index = i
    if in_block:
        if not val:
            block2_dct[index] = i
            in_block = False

cond1_blocks = [list(convert_df['Torque (mNm)'][k:v]) for k,v in block1_dct.items()]
cond2_blocks = [list(convert_df['Torque (mNm)'][k:v]) for k,v in block2_dct.items()]

For each of these, we collect the number of observations, average, and final recorded Torque of each block. We ignore blocks which are of a significantly different length to the others. That is, consider a set of $k$ blocks $b_1, b_2, ..., b_k$ of lengths $n_1, n_2, ..., n_k$ respectively. Let $$\bar{n} = k^{-1}\sum_{i=1}^k n_i$$ be the average block length. Then only blocks satisfying $$|n_i - \bar{n}| < 6$$ are used.

In [188]:
b1_counts = [len(x) for x in cond1_blocks]
b1_avgs = [np.average(x) for x in cond1_blocks]
b1_fin_vals = [x[len(x)-1] for x in cond1_blocks]
b1_to_maintain = list(abs(b1_counts-np.average(b1_counts))<6)

filt_b1_avgs = [x for x, flag in zip(b1_avgs, b1_to_maintain) if flag]
filt_b1_fin_vals = [x for x, flag in zip(b1_fin_vals, b1_to_maintain) if flag]

b2_counts = [len(x) for x in cond2_blocks]
b2_avgs = [np.average(x) for x in cond2_blocks]
b2_fin_vals = [x[len(x)-1] for x in cond2_blocks]
b2_to_maintain = list(abs(b2_counts-np.average(b2_counts))<6)

filt_b2_avgs = [x for x, flag in zip(b2_avgs, b2_to_maintain) if flag]
filt_b2_fin_vals = [x for x, flag in zip(b2_fin_vals, b2_to_maintain) if flag]


Finally for each type of block, the average of averages, standard deviation of averages, average final value, and standard deviation of final values is recorded. 

In [189]:
green_res = {
    'G Mean': np.average(filt_b1_avgs),
    'SD Gmean': np.std(filt_b1_avgs, ddof=1),
    'End Mean': np.average(filt_b1_fin_vals),
    'SD Emean': np.std(filt_b1_fin_vals, ddof=1)
}

red_res = {
    'G Mean': np.average(filt_b2_avgs),
    'SD Gmean': np.std(filt_b2_avgs, ddof=1),
    'End Mean': np.average(filt_b2_fin_vals),
    'SD Emean': np.std(filt_b2_fin_vals, ddof=1)
}

other_outputs = [
    np.max(convert_df.loc[convert_df['Cond1']]['Torque (mNm)']),
    np.average(convert_df.loc[convert_df['Cond1']]['Torque (mNm)']),
    np.max(convert_df.loc[convert_df['Cond2']]['Torque (mNm)']),
    np.average(convert_df.loc[convert_df['Cond2']]['Torque (mNm)'])
]


In [190]:
print('Green Values:')
print(green_res)
print('\n\n Red Values:')
print(red_res)

Green Values:
{'G Mean': 720.6947849314053, 'SD Gmean': 51.70469089950321, 'End Mean': 1044.78366, 'SD Emean': 108.21192399667821}


 Red Values:
{'G Mean': 76.99427724137931, 'SD Gmean': 28.9167481599348, 'End Mean': 91.837335, 'SD Emean': 133.69746636925953}


# Testing rigidity.py

Testing of the `RigidityProcessor` class is below.

In [1]:
from rigidity import RigidityProcessor

In [3]:
Rig = RigidityProcessor(filepath = 'p1350101l.txt')
Rig.get_rigidity_results()
print('Green Values:')
print(Rig.green_results)
print('\n\n Red Values:')
print(Rig.red_results)

/Users/kieranbaker/Documents/HMI-CPT-Rigidity/rigidity.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  convert_df['Channel B'] = pd.to_numeric(convert_df['Channel B'])


Green Values:
{'G Mean': 720.6947849314053, 'SD Gmean': 51.70469089950321, 'End Mean': 1044.78366, 'SD Emean': 108.21192399667821}


 Red Values:
{'G Mean': 76.99427724137931, 'SD Gmean': 28.9167481599348, 'End Mean': 91.837335, 'SD Emean': 133.69746636925953}
